In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
#batch_size = 32
#block_size = 8
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
#n_embd = 32
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

In [ ]:
# # hyperparameters
# batch_size = 32 # how many independent sequences will we process in parallel?
# block_size = 8 # what is the maximum context length for predictions?
# max_iters = 5000
# eval_interval = 500
# learning_rate = 1e-3
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# eval_iters = 200
# n_embd = 32
# n_head = 6
# n_layer = 6
# dropout = 0.2
# # ------------

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-09-16 16:11:29--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2023-09-16 16:11:29 (30.7 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
#read it to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
#print("length of dataset in char", len(text))

In [ ]:
# # all the unique characters that occur in the set
chars = sorted(list(set(text)))
vocab_size = len(chars)
# print(''.join(chars))
# print(vocab_size)

In [ ]:
# create a mapping via encoding and decoding through a dictionary

stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] #takes a string, returns the
                                        # integer for each char
decode = lambda l: ''.join([itos[i] for i in l]) #decode each number and join

# print(encode("hi there"))
# print(decode(encode("hi there")))

In [ ]:
# encode the entire dataset and store it in a torch.tensor
data = torch.tensor(encode(text), dtype=torch.long)

In [ ]:
# split the dataset into training and validation sets
n = int(0.9*len(data)) #first 90% of the dataset is training data
train_data = data[:n]
val_data = data[n:]

In [ ]:
#  block_size = 8
#  train_data[:block_size + 1]

In [ ]:
# # the possible combination examples for the above block

# x = train_data[:block_size]
# y = train_data[1:block_size + 1]
# for t in range(block_size):
#   context = x[:t+1]
#   target = y[t]
#   print(f"when input is {context} the target is {target}")

In [ ]:
torch.manual_seed(1337)
# batch_size = 4 # how many independent sequences practiced in parallel
# block_size = 8 # maximum context length for predictions

def get_batch(split):
  # generate a small batch of data of inputs x and targets y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
    # the randint function generates a list of random integers
    # which takes two arguments, the upper bound (exclusive) and
    # the shape of the tensor
    # so here, the upper bound is the length of data minus the
    # block_size which is 8, to generate a tensor of shape 4
    # meaning it will contain 4 integers
  #print(ix.tolist())
  x = torch.stack([data[i:i+block_size] for i in ix])
    # it starts at i and goes up by block size to create a tensor
    # of that many numbers in a row
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # creates a stack of the corresponding target values for inputs
  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)

print('targets:')
print(yb.shape)
print(yb)

print('=========')

# for b in range(batch_size): # batch dimension
#   for t in range(block_size):
#     context = xb[b, :t+1]
#     target = yb[b, t]
#     print(f"when input is {context.tolist()} the target is {target}")


inputs:
torch.Size([64, 256])
tensor([[ 0, 26, 53,  ..., 56, 43, 47],
        [60, 43, 56,  ..., 56,  1, 41],
        [26, 21, 33,  ..., 26, 21, 13],
        ...,
        [ 5, 57,  1,  ...,  1, 35, 47],
        [56, 53, 53,  ..., 59, 50, 42],
        [42, 47, 56,  ..., 39, 56,  1]])
targets:
torch.Size([64, 256])
tensor([[26, 53, 58,  ..., 43, 47, 45],
        [43, 56,  1,  ...,  1, 41, 53],
        [21, 33, 31,  ..., 21, 13, 10],
        ...,
        [57,  1, 52,  ..., 35, 47, 50],
        [53, 53, 58,  ..., 50, 42,  1],
        [47, 56, 43,  ..., 56,  1, 51]])


In [ ]:
#print(xb)

In [ ]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out





In [ ]:
class Head(nn.Module):
  # one head of self attention

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)
    # compute attention scores (affinities)

    wei = q @ k.transpose(-2, -1) * C**-0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    wei = self.dropout(wei)
    # perform weighted aggregation
    v = self.value(x)
    out = wei @ v
    return out

In [ ]:
class MultiHeadAttention(nn.Module):
  # multiple heads of attention in parallel

  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out


In [ ]:
class FeedForward(nn.Module):
  # a simple linear layer followed by non-linearity

  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd),
        nn.ReLU(),
        nn.Linear(4 * n_embd, n_embd),
        nn.Dropout(dropout)
    )

  def forward(self, x):
    return self.net(x)

In [ ]:
# class BatchNorm1d:

#   def __init__(self, dim, eps=1e-5, momentum=0.1):
#     self.eps = eps
#     self.gamma = torch.ones(dim)
#     self.beta = torch.ones(dim)


#   def __call__(self, x):
#     #calculate the forward pass

#     xmean = x.mean(1, keepdim=True) # batch mean
#     xvar = x.var(1, keepdim=True) # batch variance


#     xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalise
#     self.out = self.gamma * xhat + self.beta

#     return self.out

#   def parameters(self):
#     return [self.gamma, self.beta]

# torch.manual_seed(1337)
# module = BatchNorm1d(100)
# x = torch.randn(32, 100) # batch size 32 of 100 dimensional vectors
# x = module(x)
# x.shape

In [ ]:
class Block(nn.Module):
  # transformer block: comms followed by computation

  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd//n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

In [ ]:

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.ln_head = nn.Linear(n_embd, vocab_size)


  def forward(self, idx, targets=None):
    B, T = idx.shape
    #idx and targets are both (B, T) tensor of integers
    tok_embd = self.token_embedding_table(idx) # (B,T,C=n_embd) Batch x Time x Channel
    pos_embd = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
    x = tok_embd + pos_embd # (B,T,C)
    # x = self.sa_heads(x)
    # x = self.ffwd(x)
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.ln_head(x) # (B,T,C=vocab_size)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape

      logits = logits.view(B*T, C)
      targets = targets.view(B*T)

      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B,T) array of indices in the current context
    for _ in range(max_new_tokens):
      # crop idx to last block_size tokens
      idx_cond = idx[:, -block_size:]
      # get the predictions
      logits, loss = self(idx_cond)
      # focus only on the last time step
      logits = logits[:, -1, :] # becomes B, C
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # becomes B, C
      #sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # B, 1
      # append sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) # B, T + 1
    return idx

model = BigramLanguageModel()
m = model.to(device)

logits, loss = m(xb, yb)
print(logits.shape)
print(loss) # we expect -ln(1/65) = 4.17 loss but the loss we
            # get is 4.87 which means our quality of predictions
            # isnt the best


print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([16384, 65])
tensor(4.2819, grad_fn=<NllLossBackward0>)

y3,MS;KWohAJDL Z &YofWrHQjAEfolP'EAexQfr3GYs'I$Hk!;nomcXW.WoiKPzi
qU:upQl.TpLZtWBd-KhkM.G$urscpUfLdz


In [ ]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)


In [ ]:
# batch_size = 32
# for steps in range(1000):

#   #sample a batch of data
#   xb, yb = get_batch('train')

#   #evaluate the loss
#   logits, loss = m(xb, yb)
#   optimizer.zero_grad(set_to_none = True)
#   loss.backward()
#   optimizer.step()

# print(loss.item())

for iter in range(max_iters):

    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

KeyboardInterrupt: ignored

In [ ]:
#print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

In [ ]:
# # consider the following toy example

# torch.manual_seed(1337)
# B,T,C = 4,8,2 # batch, time, channels
# x = torch.randn(B,T,C)
# x.shape

In [ ]:
# # we want x[b,t] = mean_{i<=t} x[b,i]
# # version 1 of aggregation of past tokens
# xbow = torch.zeros((B,T,C))
# for b in range(B):
#   for t in range(T):
#     xprev = x[b, :t+1] # (t, C)
#     xbow[b,t] = torch.mean(xprev, 0)

In [ ]:
# # version 2 of aggregation of past tokens
# wei = torch.tril(torch.ones(T,T))
# wei = wei / wei.sum(1, keepdim=True)
# xbow2 = wei @ x # (B,T,T) @ (B,T,C) ===> (B,T,C)
# torch.allclose(xbow, xbow2)

In [ ]:
# # version 3: use Softmax
# tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
# wei = wei.masked_fill(tril == 0, float('-inf'))
# wei = F.softmax(wei, dim=-1)
# xbow3 = wei @ x
# torch.allclose(xbow, xbow3)

In [ ]:
# version 4: self-attention

torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# single head self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # B,T,16
q = query(x) # B,T,16
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> B,T,T

tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

#out = wei @ x

out.shape

In [ ]:
# torch.manual_seed(42)
# a = torch.tril(torch.ones(3, 3))
# a = a / torch.sum(a, 1, keepdim = True)
# b = torch.randint(0,10,(3,2)).float()
# c = a @ b
# print('a=')
# print(a)
# print("b=")
# print(b)
# print("c=")
# print(c)

In [ ]:
for iter in range(max_iters):

    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))